### 查看GPU

In [1]:
!nvidia-smi #查看所分配到的GPU

Thu Apr 11 00:53:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### 安裝需要的BERT套件

In [2]:
!pip install transformers==4.29.0 -U
!pip install nlp -U
!pip install torch==2.0.0 -U
!pip install transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

### 載入套件

In [3]:
from transformers import BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments
from nlp import load_dataset, Dataset
import nlp
import torch
import random
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

### 抓取訓練資料

In [4]:
!wget https://github.com/shhuangmust/AI/raw/112-1/train.csv

--2024-04-11 01:00:28--  https://github.com/shhuangmust/AI/raw/112-1/train.csv
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/shhuangmust/AI/112-1/train.csv [following]
--2024-04-11 01:00:29--  https://raw.githubusercontent.com/shhuangmust/AI/112-1/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 388895 (380K) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>] 379.78K  --.-KB/s    in 0.005s  

2024-04-11 01:00:29 (68.2 MB/s) - ‘train.csv’ saved [388895/388895]



### 讀取訓練資料集

In [5]:
df = pd.read_csv('train.csv')
dataset = Dataset.from_pandas(df) #將CSV格式轉換為資料集

In [7]:
df.sample(10) #隨機提取樣本

,ID,Name,Description,label
630,630,Childlife益生菌初乳粉末-柳橙鳳梨口味(50公克/瓶),為孩童設計的免疫與腸道健康的配方為天然柳橙、鳳梨風味的粉末型式。每份(4.3公克)提供:牛初...,0
534,534,美國許氏花旗參精萃粉光禮盒,「…花旗參最佳涼補食材醫師:夏天涼補美國威州花旗蔘首選…因為吃了參這個胃疾這個病就慢慢好了…...,1
406,406,鮑魚,「鮑魚肉有抗癌功效...鮑魚可以養陰還能平肝、固腎、效果是非常不錯的，吃鮑魚還能調整腎上腺分...,1
915,915,https://tw.buy.yahoo.com/gdsale/GNC健安喜-LAC-氣-健...,培養100%天然靈芝深層醱精粹技術而成-製程嚴密監控完全遵循GMP-螯合天然滋補之冬蟲夏草菌...,0
423,423,【FINEJAPAN】金亮Q10膠囊60粒、【FINEJAPAN】金亮綠咖啡孅盈錠150錠、...,「有效降低心血管疾病...促進免疫系統功能，改善過敏症狀…有效降低心血管，心臟病疾病…尋求預...,1
169,169,超級食物-頂級祕魯有機黑瑪卡粉（大組）,「提高性功能、提高生育能力、抵抗疲勞、保護攝護腺、對抗更年期綜合症...天然賀爾蒙發動機.....,1
836,836,康采鐵錠-avon,15毫克純鐵高效吸收純鐵高吸收率，加上完美配方維持健康，保有紅潤好氣色。,0
155,155,VeryGreenSmoothie大麥若葉酵素粉,「越來越年輕...如果要避免身體缺乏酵素，除了改善生活習慣之外，還可以從外界攝取酵素，提升免...,1
771,771,艾多美-益生菌,含有維持消化道健康的13種乳酸菌，可增加人體消化道好菌，減少壞菌滋生，促進消化道蠕動使排便順暢,0
162,162,宏醫魔力代謝B群機能咖啡補給組,「完整有酵B走肥胖效果有多好!...魔力黑咖啡防癌抗病止痛減肥防失智...維他命B群能穩定並...,1


### 載入原始模型

In [8]:
model = BertForSequenceClassification.from_pretrained('hfl/chinese-bert-wwm-ext') #載入分類模型

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of the model checkpoint at hfl/chinese-bert-wwm-ext were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkp

###  載入分詞模型

In [11]:
tokenizer = BertTokenizerFast.from_pretrained('hfl/chinese-bert-wwm-ext') #載入分詞器

### 設定執行參數

In [12]:
RANDOM_SEED = 5
MAX_LEN = 512
EPOCHS=5
BATCH_SIZE = 8

### 把資料集分成訓練、測試、驗證

In [48]:
shuffled_ds = dataset.shuffle(RANDOM_SEED) #打亂順序
split_ds = shuffled_ds.train_test_split(test_size=0.2)
train_dataset = split_ds['train']
test_val_dataset = split_ds['test']
split_tv = test_val_dataset.train_test_split(test_size=0.5)
test_dataset = split_tv['train']
val_dataset = split_tv['test']

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

### 取一筆出來瞧瞧(已經打亂)

In [49]:
train_dataset[1]

{'Description': '113種7色彩虹蔬果有酵擺脫不順蔬果增量57%，小巧好攜帶，一次完整補充一天所需的蔬果營養。',
 'ID': 819,
 'Name': '康采暢快蔬果酵素-avon',
 'label': 0}

In [50]:
tokenizer.tokenize(train_dataset[1]['Description']) #分詞

['113',
 '種',
 '7',
 '色',
 '彩',
 '虹',
 '蔬',
 '果',
 '有',
 '酵',
 '擺',
 '脫',
 '不',
 '順',
 '蔬',
 '果',
 '增',
 '量',
 '57',
 '%',
 '，',
 '小',
 '巧',
 '好',
 '攜',
 '帶',
 '，',
 '一',
 '次',
 '完',
 '整',
 '補',
 '充',
 '一',
 '天',
 '所',
 '需',
 '的',
 '蔬',
 '果',
 '營',
 '養',
 '。']

### 定義分詞器

In [51]:
def tokenize(batch): #用作取出資料集的'描述'並返回
  return tokenizer(batch['Description'], max_length=MAX_LEN, padding=True, truncation=True)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset)) #取各個資料集的'描述'
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))
val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(val_dataset))

train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label']) #更改輸出格式
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

### 定義評測準則，訓練器

In [52]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

#訓練設定
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=50,
    weight_decay=0.01,
    # evaluate_during_training=True,
    logging_dir='./logs',
)

#建立訓練器
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

### 開始訓練

In [53]:
trainer.train()
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


{'eval_loss': 0.08965492993593216,
 'eval_accuracy': 0.9893617021276596,
 'eval_f1': 0.9914529914529915,
 'eval_precision': 1.0,
 'eval_recall': 0.9830508474576272,
 'eval_runtime': 3.072,
 'eval_samples_per_second': 30.599,
 'eval_steps_per_second': 3.906,
 'epoch': 5.0}

### 找一個範例來測試

In [59]:
print(df.iloc[168]['label'], df.iloc[168]['Description'])

1 ：「血栓...心血管健康...血管無可避免的彈性變差、血液濃度變濁...在肺部就肺栓塞、在腦部就中風、在心臟則心肌梗塞、在眼睛恐造成失明，在腿部就造成下肢缺氧...預防血栓形成，並適時加入保健食品輔助...癌症病患...有心律不整者...有心房顫動問題者...有心肌梗塞病史者...有心血管家族史者...過度肥胖者...換過心臟瓣膜者...三高患者...血栓纖溶酵素於血栓預防機制...可溶解血栓的酵素，將之命名為『蚓激?』...改善了血栓造成的疾患...居高不下的血糖值也保持在正常基準值以內...紅蚯蚓具有溶解血栓的作用...溶解血栓改善動脈硬化，要心血管全面健康無慮，還必須維持血管柔軟彈性、血液循環滔滔不絕」


### 設定測試文字

In [60]:
test_text = df.iloc[168]['Description'] #取出範例描述

MAX_LEN = 512

#分詞器編碼
encoded_review = tokenizer.encode_plus(
    test_text,
    max_length=MAX_LEN,
    add_special_tokens=True,
    return_token_type_ids=False,
    return_attention_mask=True,
    return_tensors='pt',
    truncation=True
)

In [61]:
encoded_review

{'input_ids': tensor([[ 101, 8038,  519, 6117, 3410,  119,  119,  119, 2552, 6117, 5052,  978,
         2434,  119,  119,  119, 6117, 5052, 4192, 1377, 6912, 1048, 4638, 2492,
         2595, 6365, 2345,  510, 6117, 3890, 4083, 2428, 6365, 4081,  119,  119,
          119, 1762, 5511, 6956, 2218, 5511, 3410, 1853,  510, 1762, 5582, 6956,
         2218,  704, 7591,  510, 1762, 2552, 5627, 1179, 2552, 5491, 3453, 1853,
          510, 1762, 4706, 4714, 2607, 6863, 2768, 1927, 3209, 8024, 1762, 5597,
         6956, 2218, 6863, 2768,  678, 5501, 5375, 3709,  119,  119,  119, 7521,
         7344, 6117, 3410, 2501, 2768, 8024,  699, 6900, 3229, 1217, 1057,  924,
          978, 7608, 1501, 6737, 1221,  119,  119,  119, 4617, 4568, 4567, 2642,
          119,  119,  119, 3300, 2552, 2526,  679, 3146, 5442,  119,  119,  119,
         3300, 2552, 2791, 7548, 1240, 1558, 7539, 5442,  119,  119,  119, 3300,
         2552, 5491, 3453, 1853, 4567, 1380, 5442,  119,  119,  119, 3300, 2552,
         6117,

### 進行測試

In [62]:
#模型及資料設定
test_model = model.to('cuda')
input_ids = encoded_review['input_ids'].to('cuda')
attention_mask = encoded_review['attention_mask'].to('cuda')

#測試
output = test_model(input_ids, attention_mask)
result = torch.argmax(output[0][0])

#輸出結果
classnames = ['合格', '不合格']
print(f'廣告詞: {test_text}')
print(f'檢測結果: {classnames[result]}')

廣告詞: ：「血栓...心血管健康...血管無可避免的彈性變差、血液濃度變濁...在肺部就肺栓塞、在腦部就中風、在心臟則心肌梗塞、在眼睛恐造成失明，在腿部就造成下肢缺氧...預防血栓形成，並適時加入保健食品輔助...癌症病患...有心律不整者...有心房顫動問題者...有心肌梗塞病史者...有心血管家族史者...過度肥胖者...換過心臟瓣膜者...三高患者...血栓纖溶酵素於血栓預防機制...可溶解血栓的酵素，將之命名為『蚓激?』...改善了血栓造成的疾患...居高不下的血糖值也保持在正常基準值以內...紅蚯蚓具有溶解血栓的作用...溶解血栓改善動脈硬化，要心血管全面健康無慮，還必須維持血管柔軟彈性、血液循環滔滔不絕」
檢測結果: 不合格
